<a href="https://colab.research.google.com/github/mjmaher987/Stock-Prediction-Using-ML/blob/main/Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#NBEATS

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from google.colab import drive
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, Reshape, Flatten, LSTM



drive.mount('/content/drive')
# Load your DataFrame
data = pd.read_csv('/content/drive/MyDrive/EURUSD_5min.csv')

# Preprocessing
scaler = MinMaxScaler()
data['close'] = scaler.fit_transform(data['close'].values.reshape(-1, 1))

# Splitting into training and testing sets
train_size = int(0.8 * len(data))
train_data, test_data = data[:train_size], data[train_size:]


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
# Prepare data for N-BEATS
window_size = 10
X, y = [], []
for i in range(len(data) - window_size):
    X.append(data['close'][i:i+window_size])
    y.append(data['close'][i+window_size])
X = np.array(X)
y = np.array(y)

train_size = int(0.8 * len(X))
train_X, test_X = X[:train_size], X[train_size:]
train_y, test_y = y[:train_size], y[train_size:]

# Build N-BEATS model
def build_nbeats(input_shape, output_shape, num_blocks=4, num_neurons=128):
    inputs = Input(shape=input_shape)
    x = inputs
    forecasts = []
    for _ in range(num_blocks):
        for _ in range(4):  # Directly following the N-BEATS architecture
            x = Dense(num_neurons, activation='relu')(x)
        forecast = Dense(output_shape[0])(x)
        forecasts.append(forecast)
    outputs = forecasts[0]  # Use the forecast from the first block as output
    model = Model(inputs, outputs)
    return model

input_shape = (window_size, 1)
output_shape = (1,)
nbeats_model = build_nbeats(input_shape, output_shape)
nbeats_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train N-BEATS model
nbeats_model.fit(train_X, train_y, epochs=50, batch_size=32, verbose=1)

# Make predictions using the trained model
test_predictions = nbeats_model.predict(test_X)


Epoch 1/50
125/125 [==============================] - 3s 8ms/step - loss: 0.0774
Epoch 2/50
125/125 [==============================] - 1s 7ms/step - loss: 0.0591
Epoch 3/50
125/125 [==============================] - 1s 8ms/step - loss: 0.0594
Epoch 4/50
125/125 [==============================] - 1s 7ms/step - loss: 0.0594
Epoch 5/50
125/125 [==============================] - 1s 7ms/step - loss: 0.0587
Epoch 6/50
125/125 [==============================] - 1s 7ms/step - loss: 0.0586
Epoch 7/50
125/125 [==============================] - 1s 7ms/step - loss: 0.0584
Epoch 8/50
125/125 [==============================] - 1s 7ms/step - loss: 0.0585
Epoch 9/50
125/125 [==============================] - 1s 7ms/step - loss: 0.0587
Epoch 10/50
125/125 [==============================] - 1s 7ms/step - loss: 0.0586
Epoch 11/50
125/125 [==============================] - 1s 8ms/step - loss: 0.0591
Epoch 12/50
125/125 [==============================] - 1s 11ms/step - loss: 0.0583
Epoch 13/50
125/125 [===

In [6]:
reshaped_test_predictions = test_predictions[:, -1, :]

In [7]:
mae = mean_absolute_error(test_y, reshaped_test_predictions)
rmse = mean_squared_error(test_y, reshaped_test_predictions, squared=False)
print(f"Mean Absolute Error: {mae}")
print(f"Root Mean Squared Error: {rmse}")

Mean Absolute Error: 0.24873775814261148
Root Mean Squared Error: 0.26343894913971233


#NHEATS

In [8]:

X = data[['open', 'high', 'low', 'volume']].values
y = data['close'].values

train_size = int(0.7 * len(X))
val_size = int(0.15 * len(X))
train_X, test_X, train_y, test_y = X[:train_size], X[train_size:], y[:train_size], y[train_size:]
train_X, val_X, train_y, val_y = train_X[:train_size - val_size], train_X[train_size - val_size:], \
                                  train_y[:train_size - val_size], train_y[train_size - val_size:]

# Normalize data
scaler_X = MinMaxScaler()
train_X = scaler_X.fit_transform(train_X)
val_X = scaler_X.transform(val_X)
test_X = scaler_X.transform(test_X)

scaler_y = MinMaxScaler()
train_y = scaler_y.fit_transform(train_y.reshape(-1, 1)).flatten()
val_y = scaler_y.transform(val_y.reshape(-1, 1)).flatten()
test_y = scaler_y.transform(test_y.reshape(-1, 1)).flatten()

# NHITS Model
def build_nhits(input_shape):
    model = tf.keras.Sequential([
        Dense(128, activation='relu', input_shape=(input_shape,)),
        Dense(64, activation='relu'),
        Dense(1)
    ])
    return model

nhits_model = build_nhits(train_X.shape[1])
nhits_model.compile(optimizer='adam', loss='mean_squared_error')

nhits_model.fit(train_X, train_y, epochs=50, batch_size=32, validation_data=(val_X, val_y), verbose=1)

nhits_predictions = nhits_model.predict(test_X)

# Calculate MAE and RMSE for NHITS
nhits_mae = mean_absolute_error(test_y, nhits_predictions)
nhits_rmse = mean_squared_error(test_y, nhits_predictions, squared=False)
print(f"NHITS Mean Absolute Error: {nhits_mae}")
print(f"NHITS Root Mean Squared Error: {nhits_rmse}")




Epoch 1/50
86/86 [==============================] - 1s 8ms/step - loss: 0.0115 - val_loss: 1.4175e-04
Epoch 2/50
86/86 [==============================] - 0s 3ms/step - loss: 4.4874e-05 - val_loss: 3.0685e-05
Epoch 3/50
86/86 [==============================] - 0s 5ms/step - loss: 2.1515e-05 - val_loss: 2.7826e-05
Epoch 4/50
86/86 [==============================] - 0s 4ms/step - loss: 1.9430e-05 - val_loss: 2.8964e-05
Epoch 5/50
86/86 [==============================] - 0s 3ms/step - loss: 1.8558e-05 - val_loss: 2.0280e-05
Epoch 6/50
86/86 [==============================] - 0s 4ms/step - loss: 1.8451e-05 - val_loss: 1.9499e-05
Epoch 7/50
86/86 [==============================] - 0s 3ms/step - loss: 1.7363e-05 - val_loss: 1.8488e-05
Epoch 8/50
86/86 [==============================] - 0s 3ms/step - loss: 1.7405e-05 - val_loss: 2.5102e-05
Epoch 9/50
86/86 [==============================] - 1s 8ms/step - loss: 1.6817e-05 - val_loss: 1.7682e-05
Epoch 10/50
86/86 [==============================]

#RNN

In [9]:
# RNN Model
def build_rnn(input_shape):
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(64, input_shape=(input_shape, 1), activation='relu'),
        tf.keras.layers.Dense(1)
    ])
    return model

train_X_rnn = train_X.reshape(train_X.shape[0], train_X.shape[1], 1)
val_X_rnn = val_X.reshape(val_X.shape[0], val_X.shape[1], 1)
test_X_rnn = test_X.reshape(test_X.shape[0], test_X.shape[1], 1)

rnn_model = build_rnn(train_X_rnn.shape[1])
rnn_model.compile(optimizer='adam', loss='mean_squared_error')

rnn_model.fit(train_X_rnn, train_y, epochs=50, batch_size=32, validation_data=(val_X_rnn, val_y), verbose=1)


# Evaluate RNN models
rnn_predictions = rnn_model.predict(test_X_rnn)

# Calculate MAE and RMSE for RNN
rnn_mae = mean_absolute_error(test_y, rnn_predictions)
rnn_rmse = mean_squared_error(test_y, rnn_predictions, squared=False)
print(f"RNN Mean Absolute Error: {rnn_mae}")
print(f"RNN Root Mean Squared Error: {rnn_rmse}")


Epoch 1/50
86/86 [==============================] - 3s 14ms/step - loss: 0.1480 - val_loss: 0.0018
Epoch 2/50
86/86 [==============================] - 1s 8ms/step - loss: 0.0019 - val_loss: 3.5034e-04
Epoch 3/50
86/86 [==============================] - 1s 9ms/step - loss: 1.7562e-04 - val_loss: 1.1490e-04
Epoch 4/50
86/86 [==============================] - 1s 8ms/step - loss: 1.2549e-04 - val_loss: 1.1621e-04
Epoch 5/50
86/86 [==============================] - 1s 9ms/step - loss: 1.1504e-04 - val_loss: 1.2124e-04
Epoch 6/50
86/86 [==============================] - 0s 5ms/step - loss: 1.0607e-04 - val_loss: 1.2196e-04
Epoch 7/50
86/86 [==============================] - 0s 6ms/step - loss: 9.6319e-05 - val_loss: 9.1462e-05
Epoch 8/50
86/86 [==============================] - 1s 6ms/step - loss: 8.7136e-05 - val_loss: 8.4140e-05
Epoch 9/50
86/86 [==============================] - 0s 6ms/step - loss: 7.6549e-05 - val_loss: 9.0122e-05
Epoch 10/50
86/86 [==============================] - 1s 6

#LSTM

In [10]:

# LSTM Model
def build_lstm(input_shape):
    model = tf.keras.Sequential([
        LSTM(64, input_shape=input_shape, activation='relu', return_sequences=True),
        LSTM(32, activation='relu'),
        Dense(1)
    ])
    return model

train_X_lstm = train_X.reshape(train_X.shape[0], train_X.shape[1], 1)
test_X_lstm = test_X.reshape(test_X.shape[0], test_X.shape[1], 1)

lstm_model = build_lstm(train_X_lstm.shape[1:])
lstm_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

lstm_model.fit(train_X_lstm, train_y, epochs=50, batch_size=32, verbose=1)

lstm_predictions = lstm_model.predict(test_X_lstm)

# Calculate MAE and RMSE for LSTM
lstm_mae = mean_absolute_error(test_y, lstm_predictions)
lstm_rmse = mean_squared_error(test_y, lstm_predictions, squared=False)
print(f"LSTM Mean Absolute Error: {lstm_mae}")
print(f"LSTM Root Mean Squared Error: {lstm_rmse}")


Epoch 1/50
86/86 [==============================] - 5s 7ms/step - loss: 0.1358
Epoch 2/50
86/86 [==============================] - 1s 11ms/step - loss: 0.0010
Epoch 3/50
86/86 [==============================] - 1s 12ms/step - loss: 1.7437e-04
Epoch 4/50
86/86 [==============================] - 1s 12ms/step - loss: 1.1775e-04
Epoch 5/50
86/86 [==============================] - 1s 12ms/step - loss: 8.3587e-05
Epoch 6/50
86/86 [==============================] - 1s 12ms/step - loss: 6.6323e-05
Epoch 7/50
86/86 [==============================] - 1s 8ms/step - loss: 5.0098e-05
Epoch 8/50
86/86 [==============================] - 1s 7ms/step - loss: 4.1293e-05
Epoch 9/50
86/86 [==============================] - 1s 7ms/step - loss: 3.7431e-05
Epoch 10/50
86/86 [==============================] - 1s 7ms/step - loss: 3.4915e-05
Epoch 11/50
86/86 [==============================] - 1s 7ms/step - loss: 3.2976e-05
Epoch 12/50
86/86 [==============================] - 1s 7ms/step - loss: 3.2701e-05
Epoc

#Transformers

In [31]:



# def build_transformer(input_shape, num_heads=4, ff_dim=32, num_transformer_blocks=4):
#     inputs = Input(shape=input_shape)
#     x = inputs
#     for _ in range(num_transformer_blocks):
#         attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=2)(x, x)
#         attn_output = Reshape(target_shape=(-1, input_shape[1]))(attn_output)
#         x = tf.keras.layers.Add()([x, attn_output])
#         x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
#         ffn = tf.keras.Sequential([
#             tf.keras.layers.Dense(ff_dim, activation="relu"),
#             tf.keras.layers.Dense(input_shape[1])
#         ])
#         ffn_output = ffn(x)
#         x = tf.keras.layers.Add()([x, ffn_output])
#         x = tf.keras.layers.LayerNormalization(epsilon=1e-6)(x)
#     # outputs = Dense(1)(x)
#     outputs = Dense(1, activation="linear")(x)  # Linear activation for regression

#     model = Model(inputs, outputs)
#     return model


# print(train_X.shape)
# train_X_transformer = train_X.reshape(train_X.shape[0]*train_X.shape[1]//window_size, window_size, 1)
# test_X_transformer = test_X.reshape(test_X.shape[0]*test_X.shape[1]//window_size, window_size, 1)

# input_shape = (window_size, 1)
# transformer_model = build_transformer(input_shape)
# transformer_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# transformer_model.fit(train_X_transformer, train_y, epochs=50, batch_size=32, verbose=1)

# transformer_predictions = transformer_model.predict(test_X_transformer)

# # Calculate MAE and RMSE for Transformer
# transformer_mae = mean_absolute_error(test_y, transformer_predictions)
# transformer_rmse = mean_squared_error(test_y, transformer_predictions, squared=False)
# print(f"Transformer Mean Absolute Error: {transformer_mae}")
# print(f"Transformer Root Mean Squared Error: {transformer_rmse}")


# input_shape = (train_X.shape[1],)
# transformer_model = build_transformer(input_shape)
# transformer_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# transformer_model.fit(train_X, train_y, epochs=50, batch_size=32, verbose=1)

# transformer_predictions = transformer_model.predict(test_X)

# # Calculate MAE and RMSE for Transformer
# transformer_mae = mean_absolute_error(test_y, transformer_predictions)
# transformer_rmse = mean_squared_error(test_y, transformer_predictions, squared=False)
# print(f"Transformer Mean Absolute Error: {transformer_mae}")
# print(f"Transformer Root Mean Squared Error: {transformer_rmse}")




# # Custom transformer layer
# class TransformerLayer(tf.keras.layers.Layer):
#     def __init__(self, num_heads, ff_dim, rate=0.1):
#         super(TransformerLayer, self).__init__()
#         self.multi_head_attention = MultiHeadAttention(num_heads=num_heads, key_dim=2)
#         self.ffn = tf.keras.Sequential([
#             Dense(ff_dim, activation="relu"),
#             Dense(input_shape[1])
#         ])
#         self.layer_norm1 = LayerNormalization(epsilon=1e-6)
#         self.layer_norm2 = LayerNormalization(epsilon=1e-6)
#         self.dropout1 = tf.keras.layers.Dropout(rate)
#         self.dropout2 = tf.keras.layers.Dropout(rate)

#     def call(self, inputs):
#         attn_output = self.multi_head_attention(inputs, inputs)
#         attn_output = self.dropout1(attn_output, training=True)
#         out1 = self.layer_norm1(inputs + attn_output)
#         ffn_output = self.ffn(out1)
#         ffn_output = self.dropout2(ffn_output, training=True)
#         return self.layer_norm2(out1 + ffn_output)


# input_shape = (train_X.shape[1],)
# num_transformer_blocks = 4
# num_heads = 4
# ff_dim = 32

# inputs = Input(shape=input_shape)
# x = inputs
# for _ in range(num_transformer_blocks):
#     x = TransformerLayer(num_heads=num_heads, ff_dim=ff_dim)(x)

# outputs = Dense(1, activation="linear")(x)  # Linear activation for regression
# transformer_model = Model(inputs, outputs)

# transformer_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# transformer_model.fit(train_X, train_y, epochs=50, batch_size=32, verbose=1)

# transformer_predictions = transformer_model.predict(test_X)

# # Calculate MAE and RMSE for Transformer
# transformer_mae = mean_absolute_error(test_y, transformer_predictions)
# transformer_rmse = mean_squared_error(test_y, transformer_predictions, squared=False)
# print(f"Transformer Mean Absolute Error: {transformer_mae}")
# print(f"Transformer Root Mean Squared Error: {transformer_rmse}")




from tensorflow.keras.layers import Input, Dense, MultiHeadAttention, LayerNormalization, Flatten

# Custom transformer layer
class TransformerLayer(tf.keras.layers.Layer):
    def __init__(self, num_heads, ff_dim, rate=0.1):
        super(TransformerLayer, self).__init__()
        self.multi_head_attention = MultiHeadAttention(num_heads=num_heads, key_dim=4)
        self.ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(4)
        ])
        self.layer_norm1 = LayerNormalization(epsilon=1e-6)
        self.layer_norm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs):
        attn_output = self.multi_head_attention(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=True)
        out1 = self.layer_norm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=True)
        return self.layer_norm2(out1 + ffn_output)


input_shape = (4,)
num_transformer_blocks = 4
num_heads = 4
ff_dim = 32

def build_transformer(input_shape, num_transformer_blocks, num_heads, ff_dim):
    inputs = Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        attn_output = MultiHeadAttention(num_heads=num_heads, key_dim=4)(x, x)
        attn_output = LayerNormalization(epsilon=1e-6)(attn_output + x)

        ffn = tf.keras.Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(4)
        ])
        ffn_output = ffn(attn_output)
        x = LayerNormalization(epsilon=1e-6)(ffn_output + attn_output)

    x = Flatten()(x)
    outputs = Dense(1, activation="linear")(x)  # Linear activation for regression
    model = Model(inputs, outputs)
    return model

transformer_model = build_transformer(input_shape, num_transformer_blocks, num_heads, ff_dim)

transformer_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

transformer_model.fit(train_X, train_y, epochs=50, batch_size=32, verbose=1)

transformer_predictions = transformer_model.predict(test_X)

# Calculate MAE and RMSE for Transformer
transformer_mae = mean_absolute_error(test_y, transformer_predictions)
transformer_rmse = mean_squared_error(test_y, transformer_predictions, squared=False)
print(f"Transformer Mean Absolute Error: {transformer_mae}")
print(f"Transformer Root Mean Squared Error: {transformer_rmse}")

IndexError: ignored

In [23]:
print("Shape of train_X:", train_X.shape)
print("Shape of train_y:", train_y.shape)
print("Shape of test_X:", test_X.shape)
print("Shape of test_y:", test_y.shape)

Shape of train_X: (2750, 4)
Shape of train_y: (2750,)
Shape of test_X: (1500, 4)
Shape of test_y: (1500,)
